In [ ]:
from LODEGP.LODEGP import LODEGP, list_standard_models
import gpytorch
import torch
import matplotlib.pyplot as plt
from finite_difference import central_difference
import numpy as np

In [ ]:
V = matrix([[                                0,                          -400/981 ,                    x^2 + 981/200]
,[                                0                  ,        -200/981             ,    1/2*x^2 + 981/200]
,[                               -1                  ,-400/981*x^2 - 4 ,x^4 + 2943/200*x^2 + 962361/20000]])

In [ ]:
# Base ODE:     
# [x**2 + 981/(100*l1), 0, -1/l1, 0, x**2+981/(100*l2), -1/l2]

In [ ]:
var("t")
base_func = 1/(40*t)*sin(t)
p = matrix([[0],[0],[base_func]])
f = V*p
print(f)

f0 = 1/200*(200*base_func.diff(t, 2) + 981*base_func)
f1 = 1/200*(100*base_func.diff(t, 2) + 981*base_func)
f2 = 1/20000*(20000*base_func.diff(t, 4) + 294300*base_func.diff(t, 2) + 981**2*base_func)
plot(f1, (t, -10, 10), color='red')
f1

In [ ]:
# Check if both ODEs are satisfied
print((f0.diff(t, 2) + 9.81*f0 - f2))
print((f1.diff(t, 2) + 9.81/2*f1 - f2/2))

In [ ]:
START = 2r
END = 12r
COUNT = 100r
train_x = torch.linspace(START, END, COUNT)
likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=3r)

y0_func = lambda x: float(781/8000)*torch.sin(x)/x - float(1/20)*torch.cos(x)/x**2 + float(1/20)*torch.sin(x)/x**3
y1_func = lambda x: float(881/8000)*torch.sin(x)/x - float(1/40)*torch.cos(x)/x**2 + float(1/40)*torch.sin(x)/x**3
y2_func = lambda x: float(688061/800000)*torch.sin(x)/x - float(2543/4000)*torch.cos(x)/x**2 + float(1743/4000)*torch.sin(x)/x**3 - float(3/5)*torch.cos(x)/x**4 + float(3/5)*torch.sin(x)/x**5 
y0 = y0_func(train_x)
y1 = y1_func(train_x)
y2 = y2_func(train_x)
train_y = torch.stack([y0, y1, y2], dim=1r)

model = LODEGP(train_x, train_y, likelihood, 3, ODE_name="Bipendulum", verbose=True, system_parameters={"l1": 1.0, "l2": 2.0})

In [ ]:
# ODE 1
#(f0.diff(t, 2) + 9.81*f0 - f2)
h = 1e-1r
print("ODE1 median error: \t" , np.median(np.abs(central_difference(y0_func, train_x, h=h, precision=6, order=2) + 9.81r*y0_func(train_x) - y2_func(train_x))))
print("ODE1 avg. error: \t" , np.average(np.abs(central_difference(y0_func, train_x, h=h, precision=6, order=2) + 9.81r*y0_func(train_x) - y2_func(train_x))))



# ODE 2
#(f1.diff(t, 2) + 9.81/2*f1 - f2/2)

print("ODE2 median error: \t" , np.median(np.abs(central_difference(y1_func, train_x, h=h, precision=6, order=2) + 9.81r*y1_func(train_x)*0.5r - y2_func(train_x)*0.5r)))
print("ODE2 avg. error: \t" , np.average(np.abs(central_difference(y1_func, train_x, h=h, precision=6, order=2) + 9.81r*y1_func(train_x)*0.5r - y2_func(train_x)*0.5r)))

In [ ]:
# Plot the posterior GP and the data
model.eval()
model.likelihood.eval()
with torch.no_grad():
    test_x = torch.linspace(2r, 12r, 100r)
    observed_pred = model.likelihood(model(test_x))
    observed_pred_mean = observed_pred.mean
    observed_pred_var = observed_pred.covariance_matrix.diag().reshape(-1, 3)
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    for i in range( 3):
        ax.plot(test_x.numpy(), observed_pred_mean[:, i].numpy(), 'k')
        ax.fill_between(test_x.numpy(),
                        observed_pred_mean[:, i].numpy() - 1.96 * observed_pred_var[:, i].sqrt().numpy(),
                        observed_pred_mean[:, i].numpy() + 1.96 * observed_pred_var[:, i].sqrt().numpy(),
                        alpha=0.5)
        ax.scatter(train_x.numpy(), train_y[:, i].numpy(), s=10, c='r', marker='x')
    ax.legend(['Observed Data', 'Mean', 'Confidence'])
    ax.set_title('Posterior GP')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    plt.show()